In [ ]:
factory = [ "Hung Yen", "Thu Duc"]
delivery_center = ["Ha Noi", "Da Nang", "TPHCM"]
customers = [
    "Hải Phòng",
    "Thanh Hóa",
    "Lào Cai",
    "Nghệ An",
    "Quảng Ngãi",
    "Vũng Tàu",
    "Bình Dương",
    "Long An",
    "Vĩnh Long",
    "Cần Thơ"]
cost_f2d = [[0.04, 0.64, -1],
            [0.75, 0.73, 0.04]]
cost_d2d = [[-1, 0.64, 0.7],
            [-1, -1, -1],
            [0.74, 1.05, -1]]
cost_d2c = [[]]

In [12]:
import heapq

def dijkstra(graph, start, end):
    heap = [(0, start)]
    visited = set()
    distances = {start: 0}
    predecessors = {}  # Lưu trữ các đỉnh trước đỉnh hiện tại trên đường đi ngắn nhất

    while heap:
        (cost, node) = heapq.heappop(heap)

        if node == end:
            path = []
            while node is not None:
                path.append(node)
                node = predecessors.get(node)
            path.reverse()
            return [path, distances[end]]

        if node not in visited:
            visited.add(node)

            for neighbor, weight in graph[node].items():
                if neighbor not in visited:
                    new_cost = cost + weight
                    if new_cost < distances.get(neighbor, float('inf')):
                        distances[neighbor] = new_cost
                        predecessors[neighbor] = node
                        heapq.heappush(heap, (new_cost, neighbor))

    return None  # Trả về None nếu không có đường đi từ start đến end

# Bảng giá vận chuyển
shipping_costs = {
    ('P1', 'D1'): 0.04,
    ('P1', 'D2'): 0.64,
    ('P2', 'D1'): 0.75,
    ('P2', 'D2'): 0.73,
    ('P2', 'D3'): 0.04,
    ('D1', 'D2'): 0.64,
    ('D1', 'D3'): 0.7,
    ('D3', 'D2'): 1.05,
    ('D3', 'D1'): 0.74,
    ('D1', 'C1'): 0.16,
    ('D1', 'C2'): 0.24,
    ('D1', 'C3'): 0.4,
    ('D2', 'C1'): 1.12,
    ('D2', 'C4'): 0.84,
    ('D2', 'C5'): 0.2,
    ('D3', 'C1'): 2.52,
    ('D3', 'C6'): 0.12,
    ('D3', 'C7'): 0.1,
    ('D3', 'C8'): 0.08,
    ('D3', 'C9'): 0.2,
    ('D3', 'C10'): 0.24,
}

# Tạo đồ thị mới với cả khách hàng
graph = {}
for (source, dest), cost in shipping_costs.items():
    if source not in graph:
        graph[source] = {}
    if dest not in graph:
        graph[dest] = {}
    graph[source][dest] = cost

start = 'D2'
end = 'C6'
sorted_paths = dijkstra(graph, start, end)

None


In [13]:
graph

{'P1': {'D1': 0.04, 'D2': 0.64},
 'D1': {'D2': 0.64, 'D3': 0.7, 'C1': 0.16, 'C2': 0.24, 'C3': 0.4},
 'D2': {'C1': 1.12, 'C4': 0.84, 'C5': 0.2},
 'P2': {'D1': 0.75, 'D2': 0.73, 'D3': 0.04},
 'D3': {'D2': 1.05,
  'D1': 0.74,
  'C1': 2.52,
  'C6': 0.12,
  'C7': 0.1,
  'C8': 0.08,
  'C9': 0.2,
  'C10': 0.24},
 'C1': {},
 'C2': {},
 'C3': {},
 'C4': {},
 'C5': {},
 'C6': {},
 'C7': {},
 'C8': {},
 'C9': {},
 'C10': {}}

#Begin

In [133]:
import pandas as pd

# Đọc file Excel, bỏ qua hàng chứa tiêu đề "Route"
file_path = "/content/Book1.xlsx"
df = pd.read_excel(file_path, header=1)

df.columns = ["From", "To", "From_id", "To_id", "Leadtime", "Cost"]
# In ra dữ liệu trong sheet đã đọc
print(df.head())

       From       To From_id To_id  Leadtime  Cost
0  Hưng Yên   Hà Nội      P1    D1       0.3   0.3
1  Hưng Yên  Đà Nẵng      P1    D2       3.0   3.0
2   Thủ Đức   Hà Nội      P2    D1       7.0   8.0
3   Thủ Đức  Đà Nẵng      P2    D2       6.0   6.0
4   Thủ Đức   TP.HCM      P2    D3       0.3   0.3


In [274]:
import heapq

def dijkstra(graph, start, end):
    heap = [(0, start)]
    visited = set()
    distances = {start: 0}
    predecessors = {}  # Lưu trữ các đỉnh trước đỉnh hiện tại trên đường đi ngắn nhất

    while heap:
        (cost, node) = heapq.heappop(heap)

        if node == end:
            path = []
            while node is not None:
                path.append(node)
                node = predecessors.get(node)
            path.reverse()
            return [start, path, distances[end]]

        if node not in visited:
            visited.add(node)

            for neighbor, weight in graph[node].items():
                if neighbor not in visited:
                    new_cost = round(cost + weight,4)
                    if new_cost < distances.get(neighbor, float('inf')):
                        distances[neighbor] = new_cost
                        predecessors[neighbor] = node
                        heapq.heappush(heap, (new_cost, neighbor))

    return None  # Trả về None nếu không có đường đi từ start đến end

# Bảng giá vận chuyển
shipping_costs = {}
for index, row in df.iterrows():
    from_code = row['From_id']
    to_code = row['To_id']
    cost = row['Cost']
    shipping_costs[(from_code, to_code)] = cost

# Tạo đồ thị mới với cả khách hàng
graph = {}
for (source, dest), cost in shipping_costs.items():
    if source not in graph:
        graph[source] = {}
    if dest not in graph:
        graph[dest] = {}
    graph[source][dest] = cost

# Tạo đường đi ngắn nhất từ tất cả nhà máy và kho đến khách hàng
def cheapest_shipping_costs(graph, start_nodes, end_node):
    cheapest_costs = []
    for start in start_nodes:
      distances = dijkstra(graph, start, end_node)
      cheapest_costs.append(distances)
    return cheapest_costs

# start = 'D2'
# end = 'C6'
# sorted_paths = dijkstra(graph, start, end)
# print(sorted_paths)

In [139]:
def min_cost(arr):
  min = arr[0]
  for i in arr[1:]:
    if i != None:
      if min==None:
        min=i
      elif i[2]<min[2]:
        min=i
  return min

In [173]:
unique_fromid = df['From_id'].unique()
unique_fromid

array(['P1', 'P2', 'D1', 'D3', 'D2'], dtype=object)

In [279]:
import copy
def modify_graph_d(graph,warehouse1,warehouse2):
  modified_data = copy.deepcopy(graph)
  for key, value in modified_data.items():
    if key.startswith('D'):
        for inner_key in list(value.keys()):
            if inner_key.startswith('C') and key not in [warehouse1,warehouse2]:
                if inner_key in modified_data[key]:
                #print(key,inner_key)
                  del modified_data[key][inner_key]
  return modified_data



In [326]:
def solution(customer, unique_fromid, order, stock, graph):
  cost = 0
  result = []

  total_stock=sum(stock.values())
  # print(total_stock)
  if total_stock < order:
    return -1
  # print("Stock:",total_stock)

  # print(customer)
  # print("Unique:",unique_fromid)
  # print("Graph:",graph)
  #Tìm đường đi ngắn nhất từ các nhà máy, kho đến khách hàng.
#  for from_id in unique_fromid:
#    sorted_paths.append(dijkstra(graph, from_id, customer))
  sorted_paths = cheapest_shipping_costs(graph, unique_fromid, customer)
  #Nếu 1 nhà máy hoặc kho đủ hàng
  min = min_cost(sorted_paths)
  # print("Paths:",sorted_paths)
  # print(min)
  if min==None:     #Nếu không có đường nào vận chuyển đến khách hàng.
    return None
  elif stock[min[0]]>=order:
    return [min[1],order*min[2]]



  temp = []
  # print("+"*50)
  # print(sorted_paths)
  # print("+"*50)
  for i in range(len(sorted_paths)-1):
    for j in range(i+1,len(sorted_paths)):
      #Nếu bắt cặp rỗng hoặc bắt cặp với nhà máy thì bỏ qua
      if sorted_paths[i] == None or sorted_paths[j] == None:
        continue
      elif sorted_paths[i][0].startswith('P') or sorted_paths[j][0].startswith('P'):
        continue
      else:
        #print("WH:",sorted_paths[i][0],sorted_paths[j][0])
        modify_graph = modify_graph_d(graph,sorted_paths[i][0],sorted_paths[j][0])
        temp = cheapest_shipping_costs(modify_graph, unique_fromid, customer)
        result_temp = []
        total_cost = 0
        temp_order = order
        #Duyệt qua từng nhà máy, kho vận chuyển xem cái nào chi phí thấp nhất thì ưu tiên lấy hàng ở đó trước
        while temp_order>0 and len(temp)>0:
          # print("-"*50)
          # print(temp)
          # print(temp_order)
          # print(result_temp)
          # print("Len:",len(temp))
          min = min_cost(temp)      #Nơi có phí vận chuyển thấp nhất
          if min == None:     #Nếu nơi đó không có đường đi thì tiếp tục duyệt qua nơi khác
            temp.remove(min)
            continue
          elif stock[min[0]] >= temp_order:  #Nếu chỗ đó đủ hàng thì dừng lại và lưu kết quả
            result_temp.append(min[1:])
            total_cost += min[2]*stock[min[0]]
            temp_order = 0
            break
          elif stock[min[0]]<=0:    #Nếu chỗ đó không có hàng thì bỏ qua
            temp.remove(min)
            continue
          else:
            result_temp.append(min[1:])
            total_cost += min[2]*stock[min[0]]
            temp_order -= stock[min[0]]
            temp.remove(min)
        if temp_order==0:
          result.append([result_temp,total_cost])
  min = result[0]
  for i in result[1:]:
    if i[1] < min[1]:
      min = i
  #print(result)
  return min


In [327]:
stock = {}
for i in unique_fromid:
  stock[i] = 2
print(solution("C1",unique_fromid,10,stock,graph))

[[[['D1', 'C1'], 1.2], [['P1', 'D1', 'C1'], 1.5], [['D2', 'C1'], 3.0], [['P2', 'D2', 'C1'], 9.0], [['D3', 'D2', 'C1'], 9.0]], 47.4]


In [155]:
data = {
    'P1': {'D1': 0.04, 'D2': 0.64},
    'D1': {'D2': 0.64, 'D3': 0.7, 'C1': 0.16, 'C2': 0.24, 'C3': 0.4},
    'D2': {'C1': 1.12, 'C4': 0.84, 'C5': 0.2},
    'P2': {'D1': 0.75, 'D2': 0.73, 'D3': 0.04},
    'D3': {'D2': 1.05, 'D1': 0.74, 'C1': 2.52, 'C6': 0.12, 'C7': 0.1, 'C8': 0.08, 'C9': 0.2, 'C10': 0.24},
    'C1': {},
    'C2': {},
    'C3': {},
    'C4': {},
    'C5': {},
    'C6': {},
    'C7': {},
    'C8': {},
    'C9': {},
    'C10': {}
}

# Tạo một bản sao của từ điển để tránh thay đổi trong quá trình lặp
modified_data = data.copy()

for key, value in data.items():
    if 'D3' in value or key=='D3':
        del modified_data[key]

print("Từ điển sau khi xoá tất cả các phần tử có chứa 'D3':")
print(modified_data)


Từ điển sau khi xoá tất cả các phần tử có chứa 'D3':
{'P1': {'D1': 0.04, 'D2': 0.64}, 'D2': {'C1': 1.12, 'C4': 0.84, 'C5': 0.2}, 'C1': {}, 'C2': {}, 'C3': {}, 'C4': {}, 'C5': {}, 'C6': {}, 'C7': {}, 'C8': {}, 'C9': {}, 'C10': {}}


#Error

In [ ]:
import numpy as np
factory = []
delivery_center = []
customer = []


cost_f2d = np.full((2,3),-1.0) #2 nhà máy, 3 trung tâm giao hàng
cost_d2d = np.full((3,3),-1.0)
cost_d2c = np.full((3,10),-1.0) #3 trung tâm giao hàng, 10 khách hàng



In [ ]:
# for i in range(len(df)):
#   #print(i)
#   from_int = int(df["From_id"][i][1:])-1
#   to_int = int(df["To_id"][i][1:])-1
#   if df["From_id"][i][0]=="P":
#     cost_f2d[from_int][to_int] = float(df["Cost"][i])
#   elif df["To_id"][i][0]=="D":
#     cost_d2d[from_int][to_int] = float(df["Cost"][i])
#   else:
#     cost_d2c[from_int][to_int] = float(df["Cost"][i])


In [ ]:
# stock_f = np.full(2,99999999)
# stock_d = np.full(3,99999999)

In [ ]:
#import heapq
def solution(customer,stock,stock_d,stock_f,cost_f2d,cost_d2d,cost_d2c):
  cost = 0
  result = []
  cost_d2customer = [row[int(customer[1:])-1] for row in cost_d2c] #Cost từ delivery đến customer

  #Nếu kho có cost nhỏ nhất đủ hàng
  min_d2c = min_cost(cost_d2customer)
  id_d2c = cost_d2customer.index(min_d2c)
  if stock_d[id_d2c]>=stock:
      result.append([[f"D{id_d2c+1}",stock,stock*cost_d2customer[id_d2c]]])
      print(result)
      return result[0][0][2]



  #Duyệt qua từng delivery
  for i in range(len(cost_d2customer)):
    if stock_d[id_d2c]==0 or cost_d2customer==-1:
      continue
    elif stock_d[id_d2c]>=stock:
      result.append([[f"D{id_d2c+1}",stock,stock*cost_d2customer[id_d2c]]])
    else:
      result_temp = []




  #   else:
  #     stock -= stock_d[id_d2c]
  #     result_temp = []
  #     result_temp.append([f"D{id_d2c+1}",stock_d[id_d2c],stock_d[id_d2c]*cost_d2customer[id_d2c]])


  #     cost_d2delivery = [row[id_d2c] for row in cost_d2d]
  #     for j in range(len(cost_d2delivery)):
  #       min_f2d = heapq.nsmallest(i+1, cost_d2delivery)[-1]
  #       id_d2d = cost_d2delivery.index(min_f2d)
  #       if stock_d[id_d2d]==0:
  #         continue


  #   print(result)
  #   for i in result:
  #     for j in i:
  #       print(j)
  #       cost+=j[2]
  return cost

In [ ]:
def delivery_to_warehouse(warehouse,stock,stock_d,stock_f,cost_f2d,cost_d2d):
  cost_d2warehouse = [row[int(customer[1:])-1] for row in cost_d2d]
  stock_a =

In [ ]:
cost_d2c

array([[ 0.16,  0.24,  0.4 , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,
        -1.  ],
       [ 1.12, -1.  , -1.  ,  0.84,  0.2 , -1.  , -1.  , -1.  , -1.  ,
        -1.  ],
       [ 2.52, -1.  , -1.  , -1.  , -1.  ,  0.12,  0.1 ,  0.08,  0.2 ,
         0.24]])

In [ ]:
print(solution("C6",2,stock_d,stock_f,cost_f2d,cost_d2d,cost_d2c))

[[['D3', 2, 0.24]]]
0.24


In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 30.3 MB/s eta 0:00:00


In [ ]:
import pulp

# Define the problem
prob = pulp.LpProblem("Minimize_Transportation_Costs", pulp.LpMinimize)

# Hypothetical lists of entities
plants = ['P1', 'P2']
warehouses = ['D1', 'D2', 'D3']
customers = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']

# Example data structure for costs (should be provided as input in reality)
costs = {
    ('P1', 'D1'): 3, ('P1', 'D2'): 8, #... and so on for all plant to warehouse costs
    ('D1', 'C1'): 1.2, #... and so on for all warehouse to customer costs
    ('P1', 'C1'): 10, #... and so on for direct plant to customer costs if applicable
}

# Decision variables
x = pulp.LpVariable.dicts("shipment_plant_warehouse", costs.keys(), lowBound=0, cat='Integer')
y = pulp.LpVariable.dicts("shipment_warehouse_customer", costs.keys(), lowBound=0, cat='Integer')
z = pulp.LpVariable.dicts("shipment_plant_customer", costs.keys(), lowBound=0, cat='Integer')

# Objective function
prob += pulp.lpSum([costs[k] * (x[k] + y[k] + z[k]) for k in costs.keys()]), "Total Cost of Transportation"

# Constraints for customer demand
for c in customers:
    prob += pulp.lpSum([y[(d, c)] for d in warehouses] + [z[(p, c)] for p in plants]) == n, f"Demand for customer {c}"

# Constraint for maximum warehouses per customer
for c in customers:
    prob += pulp.lpSum([pulp.lpSum([y[(d, c)] for d in warehouses]) > 0 for d in warehouses]) <= 2, f"Max warehouses for customer {c}"

# Solve the problem
prob.solve()

# Output the results
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

# Print the optimized total cost
print("Total Cost of Transportation =", pulp.value(prob.objective))


KeyError: ('D2', 'C1')